# Base de datos Riak

In [1]:
import riak
from pprintpp import pprint as pp
import json
import pandas as pd

import uuid
import hashlib

## Conexion a RIAK

In [2]:
# connect to database
cliente = riak.RiakClient()
cliente.ping()

True

## Funciones utiles para el desarrollo de la práctica

In [3]:
# ELimina datos de un Bucket
def drop_keys(bucket):
    for keys in bucket.stream_keys():
        # print(keys)
        for keys in bucket.stream_keys():
            for key in keys:
                bucket.delete(key)

In [4]:
# Devuelve el hash de un string
def hash_string(s):
    s_utf8 = s.encode('utf-8')
    return hashlib.md5(s_utf8).hexdigest()

In [5]:
# Convertimos el objeto JSON en un objeto pandas 
def json_to_pandas(jsonObj):
    return pd.read_json(json.dumps(jsonObj))

## Lectura de la informacion en Pandas

In [6]:
df_dificultades = pd.read_csv("./data/dificultades.csv",encoding='utf-8');
df_encadenamientos = pd.read_csv("./data/tipos_encadenamiento.csv", encoding='utf-8');
df_escaladores = pd.read_csv("./data/escaladores_lite_2017.csv", encoding='utf-8');
df_ascensos = pd.read_csv("./data/ascensos_lite_2017.csv", encoding='utf-8');

In [7]:
# 'merge' del tipo de encadenamiento en el data frame de ascensos
df_ascensos = pd.merge(df_ascensos, df_encadenamientos, on = ['id_tipo_encadenamiento'], how = 'inner')
df_ascensos = df_ascensos[['id_escalador','id_dificultad', 'tipo_encadenamiento', 'nombre_via', 'risco', 'sector', 'pais']];


## Desnormalización del data frame de ascensos

Voy a incluir la información del usuario en cada registro de ascenso, para ir creando los buckets que van interesando según las preguntas a resolver


In [8]:
df_ascensos_desnormalizado = pd.merge(df_ascensos, df_escaladores, on = ['id_escalador'], how = 'inner');
df_ascensos_desnormalizado = pd.merge(df_ascensos_desnormalizado, df_dificultades, on = ['id_dificultad'], how = 'inner');

#renombrar campos autonombrados por merge
df_ascensos_desnormalizado = df_ascensos_desnormalizado.rename(
    columns={
        'pais_x' : 'pais_risco',
        'ciudad':'ciudad_escalador',
        'pais_y':'pais_escalador'});

df_ascensos_desnormalizado.head()

,id_escalador,id_dificultad,tipo_encadenamiento,nombre_via,risco,sector,pais_risco,nombre,sexo,fecha_nacimiento,ciudad_escalador,pais_escalador,comienzo,grado_frances
0,20384,33,Flash,LE DE,FONTAINEBLEAU,ROCHER CANON,FRA,Clemens Kurth,Hombre,1982-04-24,Alfeld (Leine),DEU,2003,5c
1,20384,33,Flash,SCHMALSEITE DV,ELBSANDSTEIN,KRALLENTURM,DEU,Clemens Kurth,Hombre,1982-04-24,Alfeld (Leine),DEU,2003,5c
2,20384,33,Onsight,TALSEITE,ELBSANDSTEIN,FöRSTER,DEU,Clemens Kurth,Hombre,1982-04-24,Alfeld (Leine),DEU,2003,5c
3,20384,33,Redpoint,LA DALLE AU PERNOD,FONTAINEBLEAU,BAS CUVIER,FRA,Clemens Kurth,Hombre,1982-04-24,Alfeld (Leine),DEU,2003,5c
4,20384,33,Redpoint,TALSEITE,HARZVORLAND / LANGENSTEIN,BLOCK 17,DEU,Clemens Kurth,Hombre,1982-04-24,Alfeld (Leine),DEU,2003,5c


## PREPARACIÓN DE LA BASE DE DATOS

Para resolver las preguntas me interesará tener los siguientes buckets:

|bucket|tipo|clave|Contenido del bucket|
|-|-|-|
|dificultades|default|id_dificultad|dificicultad en grado frances, usa y bloque-usa|
|ascensos|default|generada con identificador unico|Información desnormalizada de cada ascenso|
|ascensos_acum|'map'|id_escalador|Un mapa por escalador con: nombre, sexo, pais, ascensos_acumulados, suma_dificultades, , dificultad maxima en españa, ascensos acumulados en españa y suma dificultad en España|
|riscos_acum|'map'|hash del nombre del risco|Un mapa por risco: nombre, pais, suma_ascensos, suma_dificultades, dificultad maxima|

** Indices por bucket **

|Bucket|Nombre del índice|Contenido|¿Que busquedas permite hacer?|
|-|-|-|-|
|ascensos|idx_IdEscalador_int|Id del escalador|Localizar movimientos de un escalador concreto|



**CREACIÓN DE JSON CON LA INFORMACIÓN QUE IRÁ A LOS BUCKETS**

In [9]:
ascensosJsonString = df_ascensos_desnormalizado.to_json(orient = 'records')
ascensosJson = json.loads(ascensosJsonString)

In [10]:
dificultadesJsonString = df_dificultades.to_json(orient = 'records')
dificultadesJson = json.loads(dificultadesJsonString)

**CREACIÓN DE BUCKET ASCENSOS ACUMULADOS**

In [11]:
BUCKET_ACUM_ASCENSOS = 'acum_ascensos'
bucket_acum_ascensos = cliente.bucket_type('maps').bucket(BUCKET_ACUM_ASCENSOS)
drop_keys(bucket_acum_ascensos)



for ascensoJson in ascensosJson:
    # construcción de ascensos acumulados por escalador
    acumulador_ascensos = bucket_acum_ascensos.new(str(ascensoJson['id_escalador']))
    acumulador_ascensos.registers['nombre'].assign(ascensoJson['nombre'].encode('utf-8'))
    acumulador_ascensos.registers['sexo'].assign(ascensoJson['sexo'].encode('utf-8'))
    acumulador_ascensos.registers['pais'].assign(ascensoJson['pais_escalador'].encode('utf-8'))
    acumulador_ascensos.counters['numero_ascensos'].increment()      
    acumulador_ascensos.counters['suma_dificultades'].increment(ascensoJson['id_dificultad'])
    
    if (ascensoJson['pais_risco'] == 'ESP') :
        
        str_dificultad_maxima_registrada = acumulador_ascensos.registers['id_dificultad_max'].value  
        dificultad_maxima = ascensoJson['id_dificultad']
        
        if (str_dificultad_maxima_registrada != ''):
            dificultad_maxima = max(int(str_dificultad_maxima_registrada), ascensoJson['id_dificultad'])
            
        acumulador_ascensos.registers['id_dificultad_max_esp'].assign(str(dificultad_maxima))
        acumulador_ascensos.counters['numero_ascensos_esp'].increment()
        acumulador_ascensos.counters['suma_dificultades_esp'].increment(ascensoJson['id_dificultad'])
        
    acumulador_ascensos.store()
    
    
    

**CREACIÓN DE BUCKET RISCOS ACUMULADOS**

In [12]:
BUCKET_ACUM_RISCOS = 'acum_riscos'
bucket_acum_riscos = cliente.bucket_type('maps').bucket(BUCKET_ACUM_RISCOS)
drop_keys(bucket_acum_riscos)


for ascensoJson in ascensosJson:
    acumulador_risco = bucket_acum_riscos.new(hash_string(ascensoJson['risco']))
    
    str_dificultad_maxima_registrada = acumulador_risco.registers['id_dificultad_max'].value
    dificultad_maxima = ascensoJson['id_dificultad']
    if (str_dificultad_maxima_registrada != ''):
        dificultad_maxima = max(int(str_dificultad_maxima_registrada), ascensoJson['id_dificultad'])
    
    acumulador_risco.registers['risco'].assign(ascensoJson['risco'].encode('utf-8'))
    acumulador_risco.registers['pais'].assign(ascensoJson['pais_risco'].encode('utf-8'))
    acumulador_risco.registers['id_dificultad_max'].assign(str(dificultad_maxima))
        
    acumulador_risco.counters['numero_ascensos'].increment()
    acumulador_risco.counters['suma_dificultades'].increment(ascensoJson['id_dificultad'])
        
    acumulador_risco.store()

**CREACIÓN DE BUCKET ASCENSOS**

In [13]:
BUCKET_ASCENSOS = 'ascensos'
bucket_ascensos = cliente.bucket(BUCKET_ASCENSOS)
drop_keys(bucket_ascensos)

for ascensoJson in ascensosJson:
    key = uuid.uuid1().hex
    ascenso = bucket_ascensos.new(key, ascensoJson)
    ascenso.add_index('idx_IdEscalador_int', ascensoJson["id_escalador"])
    ascenso.store()

**CREACIÓN DE BUCKET DIFICULTADES**

In [14]:
BUCKET_DIFICULTADES = 'dificultades'
bucket_dificultades = cliente.bucket(BUCKET_DIFICULTADES)
drop_keys(bucket_dificultades)

for dificultadJson in dificultadesJson:
    dificultad = bucket_dificultades.new(str(dificultadJson["id_dificultad"]), dificultadJson)
    dificultad.store()

## LECTURA DE LOS BUCKETS RIAK

**DATA FRAME RISCOS ACUMULADOS**

In [15]:
listaJson = []
for streamKeys in bucket_acum_riscos.stream_keys():
    for streamKey in streamKeys:
        acumulador_riscos = bucket_acum_riscos.get(streamKey)
        listaJson.append({'risco' : acumulador_riscos.registers['risco'].value,
                          'pais' : acumulador_riscos.registers['pais'].value,
                          'id_dificultad_max' : acumulador_riscos.registers['id_dificultad_max'].value,
                          'numero_ascensos' : acumulador_riscos.counters['numero_ascensos'].value,
                          'suma_dificultades' : acumulador_riscos.counters['suma_dificultades'].value})
        
# Convertimos el objeto json en un objeto pandas 
df_bucket_riscos_acumulados = json_to_pandas(listaJson)

La columna de identificacion de dificultad maxima viene de un registro por lo que es un string. Debemos convertirlo en una columna de enteros:

In [16]:
df_bucket_riscos_acumulados['id_dificultad_max'] = pd.to_numeric(df_bucket_riscos_acumulados['id_dificultad_max'])
df_bucket_riscos_acumulados['id_dificultad_max'].fillna(value=0, inplace =True)
df_bucket_riscos_acumulados['id_dificultad_max'] = df_bucket_riscos_acumulados['id_dificultad_max'].astype(int)
df_bucket_riscos_acumulados.head(10)

,id_dificultad_max,numero_ascensos,pais,risco,suma_dificultades
0,38,2,UKR,VINNITSA,84
1,53,1,NZL,CASTLE HILL,53
2,62,1,ITA,CAPRE AL SOLE,62
3,53,1,MLT,GHAR LAPSI,53
4,31,2,POL,NIELEPICE,64
5,40,1,BRA,PLATô DA LAGOA,40
6,51,1,CHE,GORGES DU COURT,51
7,49,3,HRV,KARIN,118
8,57,1,AUT,GöLTSCHACH,57
9,44,1,ESP,OGASSA,44


**DATA FRAME ASCENSOS ACUMULADOS**

In [17]:
listaJson = []
for streamKeys in bucket_acum_ascensos.stream_keys():
    for streamKey in streamKeys:
        acumulador_ascensos = bucket_acum_ascensos.get(streamKey)
        listaJson.append({'id_escalador' : streamKey,
                          'nombre' : acumulador_ascensos.registers['nombre'].value,
                          'sexo' : acumulador_ascensos.registers['sexo'].value,
                          'pais' : acumulador_ascensos.registers['pais'].value,
                          'numero_ascensos' : acumulador_ascensos.counters['numero_ascensos'].value,
                          'suma_dificultades' : acumulador_ascensos.counters['suma_dificultades'].value,
                          'numero_ascensos_esp' : acumulador_ascensos.counters['numero_ascensos_esp'].value,
                          'suma_dificultades_esp' : acumulador_ascensos.counters['suma_dificultades_esp'].value,
                          'id_dificultad_max_esp' : acumulador_ascensos.registers['id_dificultad_max_esp'].value})
        
# Convertimos el objeto json en un objeto pandas 
df_bucket_ascensos_acumulados = json_to_pandas(listaJson)


La columna de identificacion de dificultad maxima viene de un registro por lo que es un string. Debemos convertirlo en una columna de enteros:

In [18]:
df_bucket_ascensos_acumulados['id_dificultad_max_esp'] = pd.to_numeric(df_bucket_ascensos_acumulados['id_dificultad_max_esp'])
df_bucket_ascensos_acumulados['id_dificultad_max_esp'].fillna(value=0, inplace =True)
df_bucket_ascensos_acumulados['id_dificultad_max_esp'] = df_bucket_ascensos_acumulados['id_dificultad_max_esp'].astype(int)
#df_bucket_ascensos_acumulados.head(10)

**DATA FRAME DIFICULTADES**

In [19]:
filasDificultades = []
for keys in bucket_dificultades.stream_keys():
    for key in keys:
        filasDificultades.append(bucket_dificultades.get(key).data) 
df_bucket_dificultades = json_to_pandas(filasDificultades);
df_bucket_dificultades.head(3)

,grado_frances,id_dificultad
0,8a/+,63
1,5c,33
2,4c,25


Como se observa, RIAK no devuelve orden, pero en nuestro caso el orden de dificultad viene dado por id_dificultad. Para facilitar posteriores mapeos, es interesante ordenar este data frame y restablecer el indice

In [20]:
df_bucket_dificultades = df_bucket_dificultades.sort_values('id_dificultad',ascending=True);
df_bucket_dificultades=df_bucket_dificultades.set_index('id_dificultad');
df_bucket_dificultades.head(3)

,grado_frances
id_dificultad,
1,-
2,1
3,1a


## RESPUESTAS A LAS PREGUNTAS

### 1.a) Los 10 escaladores (hombres) más activos. (orden auxiliar por Id)

In [21]:
df = df_bucket_ascensos_acumulados[df_bucket_ascensos_acumulados['sexo']=='Hombre'];
df = df.sort_values(['numero_ascensos', 'id_escalador'],ascending=[False, False]);
df[['id_escalador', 'nombre', 'sexo', 'pais', 'numero_ascensos']].head(10)

,id_escalador,nombre,sexo,pais,numero_ascensos
3020,50884,Christopher Leonetti,Hombre,USA,47
2527,20384,Clemens Kurth,Hombre,DEU,26
2022,20095,Matthias Schuster,Hombre,DEU,26
4063,66250,Kuba Kaminski,Hombre,POL,24
4391,9171,Laurenz Trawnicek,Hombre,AUT,23
1636,46622,Raúl Crespo,Hombre,ESP,19
530,42086,Marcin Opozda,Hombre,POL,19
4550,66466,Christian Boehme,Hombre,DEU,18
1007,47732,Thomas de Fleurian,Hombre,FRA,18
1374,35847,Zack Bum,Hombre,DEU,18


In [22]:
escalador_mas_activo = df['id_escalador'].iloc[0]
print ('Id del escalador mas activo', escalador_mas_activo)

('Id del escalador mas activo', 50884)


### 1.b) Los 10 escaladoras (mujeres) más activas. (orden auxiliar por Id)

In [23]:
df = df_bucket_ascensos_acumulados[df_bucket_ascensos_acumulados['sexo']=='Mujer'];
df = df.sort_values(['numero_ascensos', 'id_escalador'],ascending=[False, False]);
df[['id_escalador', 'nombre', 'sexo', 'pais', 'numero_ascensos']].head(10)

,id_escalador,nombre,sexo,pais,numero_ascensos
3546,65502,Ksenia Targosz,Mujer,POL,13
3744,65069,Mania Mania A.,Mujer,POL,12
3339,65707,Ewelina Cienkus,Mujer,POL,11
2998,54695,Elfi Hasler,Mujer,AUT,11
3273,62354,La Shoune,Mujer,FRA,10
3794,58835,Dominika Sołtys,Mujer,POL,10
1497,57447,Sue Murphy,Mujer,CAN,10
3811,53983,Daniela Bärtschi,Mujer,CHE,10
4286,49569,Karina Kosiorek,Mujer,POL,10
2295,32043,Reidun M. Romundstad,Mujer,NOR,10


In [24]:
escaladora_mas_activa = df['id_escalador'].iloc[0]
print ('Id de la escaladora mas activa', escaladora_mas_activa)

('Id de la escaladora mas activa', 65502)


### 2. Lista de los 10 ascensos "On sight" de la escaladora más activa en orden decreciente de dificultad (y por nombre de vía ascendete)

En este caso voy a obtener los **ascensos de la escaladora más activa** (de la pregunta anterior se que id_escalador=53983) **accediendo por el indice al bucket de ascensos**:

In [25]:
filasAscensos = []
streamIndex = bucket_ascensos.stream_index("idx_IdEscalador_int", escaladora_mas_activa)
for keys in streamIndex.results:
    for key in keys:
        filasAscensos.append(bucket_ascensos.get(key).data)
    
df_bucket_ascensos = json_to_pandas(filasAscensos);

In [26]:
df_bucket_ascensos = df_bucket_ascensos[df_bucket_ascensos['tipo_encadenamiento']=='Onsight']
df_bucket_ascensos = df_bucket_ascensos.sort_values(['id_dificultad', 'nombre_via'],ascending=[False, True])
df_bucket_ascensos[['nombre_via', 'grado_frances', 'risco', 'pais_risco']].head(10)


,nombre_via,grado_frances,risco,pais_risco
0,LEWIZNA,5c,DOLINA KOBYLANSKA,POL
1,MOJA KOKAINA,5c,DOLINA KOBYLANSKA,POL
8,RUCHY MASOWE,5c,DOLINA BEDKOWSKA,POL
7,FISCHER-LAPINSKI,5b,DOLINA BEDKOWSKA,POL
11,GRZBIET ZUBRA,5b,SOKOLIKI,POL
6,RYSKOWIEC,5b,DOLINA BEDKOWSKA,POL
10,ZAGłADA TRADA,4c,DOLINA KOBYLANSKA,POL
4,KIKUT,4b,DOLINA SZKLARKI,POL


### 3. Dificultad media de los ascensos del escalador más activo

De la pregunta 1 ya tenemos un data frame con los datos acumulados por escalador. Utilizandolo podemos obtener la media de los ascensos del escalador más activo (id_escalador = 50884):

In [27]:
df=df_bucket_ascensos_acumulados[df_bucket_ascensos_acumulados['id_escalador']==escalador_mas_activo]
id_dificultad_media = int(round(df['suma_dificultades']/df['numero_ascensos'],0))

print('Dificultad media de ascensos escalador mas activo:')
df_bucket_dificultades['grado_frances'][id_dificultad_media]


Dificultad media de ascensos escalador mas activo:


u'5c+'

### 4.a) Los 10 ascensos mas dificiles

**Esta pregunta no la podemos respoder con la base de datos**. No obstante, si podemos hacerla mediante un proceso de python. La pega que tiene es que debemos construir un data frame con todos los ascensos, lo que puede suponer traer mucha información desde la base de datos. Una posible solución es paralelizar la clasificación en trozos y despues unirlos.

El proceso de python para responder esta pregunta es:

In [28]:
filasAscensos = []

for keys in bucket_ascensos.stream_keys():
    for key in keys:
        filasAscensos.append(bucket_ascensos.get(key).data)
    
df_bucket_ascensos = json_to_pandas(filasAscensos);

In [29]:
df_bucket_ascensos = df_bucket_ascensos.sort_values(['id_dificultad', 'id_escalador'], ascending=[False, True]);
columnas = ['id_escalador','nombre','pais_escalador','nombre_via','grado_frances','tipo_encadenamiento','risco','pais_risco'];
df_bucket_ascensos[columnas].head(10)

,id_escalador,nombre,pais_escalador,nombre_via,grado_frances,tipo_encadenamiento,risco,pais_risco
10775,1476,Adam Ondra,CZE,SILENCE,9c,Redpoint,FLATANGER,NOR
10045,1476,Adam Ondra,CZE,MOVE HARD,9b,Redpoint,FLATANGER,NOR
8477,22437,Stefano Ghisolfi,ITA,FIRST ROUND FIRST MINUTE,9b,Redpoint,MARGALEF,ESP
8479,1476,Adam Ondra,CZE,ULTIMATUM,9a+,Redpoint,ARCO,ITA
10032,1476,Adam Ondra,CZE,NATURALMENTE,9a+,Redpoint,CAMAIORE,ITA
10290,8707,Daniel Fuertes,ESP,NO PAIN NO GAIN,9a+,Redpoint,RODELLAR,ESP
11339,14130,David Firnenburg,DEU,LA RAMBLA,9a+,Redpoint,SIURANA,ESP
8478,18008,Piotr Schab,POL,THOR'S HAMMER,9a+,Redpoint,FLATANGER,NOR
10031,22437,Stefano Ghisolfi,ITA,LA RAMBLA,9a+,Redpoint,SIURANA,ESP
10848,22437,Stefano Ghisolfi,ITA,FIRST LEY,9a+,Redpoint,MARGALEF,ESP


### 4.b) Los 10 ascensos mas dificiles a vista (On sight)

Aprovechando el data frame ordenado anterior, si hacemos uno nuevo cuyas filas tengan los elementos con tipo_encadenamiento igual a Onsight, hemos respondido a esta pregunta facilmente:

In [30]:
df_ascensos_onsight = df_bucket_ascensos[df_bucket_ascensos['tipo_encadenamiento']=='Onsight']
df_ascensos_onsight[columnas].head(10)

,id_escalador,nombre,pais_escalador,nombre_via,grado_frances,tipo_encadenamiento,risco,pais_risco
11095,22437,Stefano Ghisolfi,ITA,FISH EYE,8c,Onsight,OLIANA,ESP
10046,18008,Piotr Schab,POL,PEQUENA ESTRELLA,8b+,Onsight,RODELLAR,ESP
10708,6726,Manu Lopez,FRA,WHAT,8b,Onsight,LEONIDIO,GRC
11315,38626,luis rodriguez martin,ESP,BRUJO,8b,Onsight,SADERNES,ESP
10643,1476,Adam Ondra,CZE,MATA HARI,8a+,Onsight,FRANKENJURA,DEU
11435,4424,Marcin Wszolek,POL,NUEVE ZETA,8a+,Onsight,CHULILLA,ESP
10844,16672,Gonzalo Larrocha,ESP,DNA EXTENSION,8a+,Onsight,KALYMNOS,GRC
10300,27822,Michaela Kiersch,USA,LA FEMME BLANCHE,8a+,Onsight,CéüSE,FRA
7992,34114,jose luis palao,ESP,TEAM BTR,8a+,Onsight,BIELSA,ESP
10781,34114,jose luis palao,ESP,REALIDAD VIRTUAL,8a+,Onsight,POLORIA,ESP


### 5.a) Grado medio y maximo de los ascensos en España de los 10 escaladores NO ESPAÑOLES con mas ascensos en España

La base de datos no da funcionalidad para resolver esta pregunta de manera directa. Para responder a la pregunta, o se procesan los ascensos del bucket de ascensos o se registra a priori información en el bucket de ascensos acumulados que permita calculos posteriore.

En este caso yo he elegido registrar información en el bucket de ascensos acumulados la información necesaria ello se debe haber registrado en el bucket de acumulados informacion a priori acerca del numero de vias escaladas en España por los escaladores. En este caso, esa información la he construido en la creación del bucket anteriormente

In [31]:
#del data frame de ascensos acumulados me quedo solo con los escaladores que tienen ascensos en españa
df = df_bucket_ascensos_acumulados[df_bucket_ascensos_acumulados['numero_ascensos_esp']>0]
df['id_dificultad_media_esp'] = (df['suma_dificultades_esp']/df['numero_ascensos_esp']).round().astype(int);

df['grado_medio_esp']=df['id_dificultad_media_esp'].map(df_bucket_dificultades['grado_frances']);

df['grado_maximo_esp']=df['id_dificultad_max_esp'].map(df_bucket_dificultades['grado_frances']);

#ordeno el data frame del bucket de acumulados por numero de ascensos y por nombre
df = df.sort_values(['numero_ascensos_esp', 'id_escalador'], ascending=[False, True]);


In [32]:
columnas = ['nombre', 'pais', 'numero_ascensos_esp', 'grado_medio_esp', 'grado_maximo_esp']
df_NO_ESP = df[df['pais']!='ESP'];
df_NO_ESP[columnas].head(10)

,nombre,pais,numero_ascensos_esp,grado_medio_esp,grado_maximo_esp
1162,Nuno Henriques,PRT,14,6a+,6a+
2712,"Grzegorz ""Buła"" Golowczyk",POL,9,7b+,8b+
1973,philipp kieffer,DEU,7,6c+/7a,6c
4191,Wojtek Pełka,POL,7,7c+/8a,8a
1411,"Gonçalo ""Gongas"" Coutinho",PRT,6,5,4a
1078,Tieme van Veen,NLD,6,6c+/7a,8a
4293,Kuba Pe,POL,6,7a,6c
3820,Gabriel Korbiel,POL,6,7a+/7b,7a+
3278,Amber Thornton,GBR,6,5c,6c
3561,Benjamin Thomas,FRA,6,7a,7c+


### 5.b) Grado medio y maximo de los ascensos en España de los 10 escaladores ESPAÑOLES con mas ascensos en España

In [33]:
columnas = ['nombre', 'pais', 'numero_ascensos_esp', 'grado_medio_esp', 'grado_maximo_esp']
df_ESP = df[df['pais']=='ESP'];
df_ESP[columnas].head(10)

,nombre,pais,numero_ascensos_esp,grado_medio_esp,grado_maximo_esp
1636,Raúl Crespo,ESP,18,6a+/6b,7a+
4377,Chaken Gómez conde,ESP,16,7c+,8a+
320,Adrian Alameda,ESP,14,6c+,8a+
3282,Alex Garriga,ESP,13,7b+/7c,8a+
262,Xavier Gatell Romero,ESP,12,7b+,8a
2874,jose luis palao,ESP,12,8a+,8b+
239,Jose Agustí,ESP,10,7c+,8a
1315,Gonzalo Larrocha,ESP,10,8a+/8b,8b+
2387,Tomata Tomata,ESP,10,6c+,7a+
1571,luis rodriguez martin,ESP,10,8a/+,8b+


### 6.a) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con menos de 3 años de experiencia

### 6.b) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con entre 10 y 30 años de experiencia

Estas dos preguntas **no se pueden responder de manera directa con la base de datos RIAK**. Requiere procesamiento, ya sea previo a la hora de introducir la información en la base de datos, ya sea a la hora de llevar información a la base de datos como después de su obtención.

Una **posible solución alternativa a trabajar intensivamente en el lado de python con el Data Frame de todos los datos de ascensos** podría ser:

- Plantear dos nuevos buckets tipo mapa con acumuladores, uno para los escaladores de menos de 3 años y otro para los escaladores de entre 10 y 30 años de experiencia.

- En cada bucket tener por cada escalador, registros de su información y acumuladores que permitan llevar el total de ascensos no TopRope asi como la suma de las dificultades de estos ascensos.

- Rellenariamos esos buckets recorriendo el DataFrame como hemos hecho anteriormente.

- Con esos dos nuevos buckets hariamos un proceso similar al realizado en las anteriores preguntas:
    - Leeriamos el contenido del bucket llevandolo a un pandas
    - Calculariamos las medias con los valores acumulados de los buckets
    - Ordenariamos y filtrariamos en base a los requerimientos de la pregunta.



### 7. Los 10 riscos españoles (o zonas) con mas ascensos por orden decreciente de numero de ascensos

Utilizando el data frame obtenido del bucket de riscos_acumulados:

In [34]:
df_bucket_riscos_acumulados = df_bucket_riscos_acumulados.sort_values(['numero_ascensos', 'risco'], ascending=[False,True])

df_riscos_esp = df_bucket_riscos_acumulados[df_bucket_riscos_acumulados['pais']=='ESP'];

columnas = ['risco', 'numero_ascensos']
df_riscos_esp.head(10)[columnas]

,risco,numero_ascensos
1243,MARGALEF,194
1184,CUENCA,148
1573,SIURANA,147
448,ALBARRACíN,137
1851,RODELLAR,136
323,CHULILLA,134
63,MALLORCA,63
1315,LA PEDRIZA,61
340,MONTSERRAT,58
459,EL CHORRO,44


### 8.a) Los 10 sectores españoles con mayor nivel de difcultad media de ascensos ordenadas por orden decreciente de dificultad y por numero de ascensos decreciente

Con el data frame obtenido del bucket de riscos acumulados podemos construir esta informacion media que se solicita. El procesamiento de construccion de la media, seleccion y filtrado se debe hacer fuera ya que no es funcionalidad que de la base de datos:

In [35]:
df_riscos_esp['id_dificultad_media'] = (df_riscos_esp['suma_dificultades']/df_riscos_esp['numero_ascensos']).round().astype(int);
df_riscos_esp['dificultad_media']=df_riscos_esp['id_dificultad_media'].map(df_bucket_dificultades['grado_frances']);
#df_riscos_esp.head()

In [36]:
columnas = ['risco', 'id_dificultad_media', 'dificultad_media', 'numero_ascensos']
df_riscos_esp.sort_values(['id_dificultad_media', 'numero_ascensos', 'risco'], ascending=[False, False, True]).head(10)[columnas]

,risco,id_dificultad_media,dificultad_media,numero_ascensos
1726,LA COVA DE L'OCELL,70,8c,2
1564,POLORIA,66,8b,2
1516,FIGOLS,66,8b,1
967,ORIHUELA,65,8a+/8b,2
1123,SANTA LINYA,64,8a+,18
1375,OLIANA,64,8a+,14
536,FORADA,64,8a+,3
1794,LA MUELA,64,8a+,2
1739,BALTZOLA,64,8a+,1
300,CACIN,64,8a+,1


### 8.b) Las 10 sectores españoles con menor nivel de dificultad medio de ascensos ordenadas por orden creciente de dificultad y por numero de ascensos decreciente

In [37]:
df_riscos_esp.sort_values(['id_dificultad_media', 'numero_ascensos', 'risco'], ascending=[True, False, True]).head(10)[columnas]

,risco,id_dificultad_media,dificultad_media,numero_ascensos
379,PORTOMARIN,21,4a,2
1747,MONT-RAL,25,4c,1
1529,OTO,25,4c,1
671,PUERTO ROQUE,26,4c+,3
352,PUEBLA DE GUZMAN,27,4+,5
325,BALNEARIO DE PANTICOSA,27,4+,2
552,SAVASSONA,28,5,2
1766,LA DEHESA DE LA OLIVA,29,5a,1
1927,LA PUENTE,29,5a,1
678,MALLORKA,29,5a,1
